In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load data
fire_df = pd.read_csv('Fire_data.csv')

######CLEAN DATA##########

# only get rows in Butte, CA
fire_df = fire_df[fire_df['state']=='CA']
fire_df = fire_df[(fire_df['latitude']>=39.0) & (fire_df['latitude']<=41.0)]

# Remove uneccesary rows
fire_df = fire_df[['latitude', 'longitude', 'state', 'disc_clean_date', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_15', 'Wind_pre_7','Hum_pre_15', 'Hum_pre_7','Prec_pre_15', 'Prec_pre_7']]
fire_df = fire_df.drop(fire_df[fire_df['Prec_pre_15']==-1].index)
fire_df = fire_df.drop(fire_df[fire_df['Prec_pre_7']==-1].index)

# Target row (fire): 1=Fire, 0=no fire
fire_df['fire']=1
print(fire_df)

        latitude   longitude state disc_clean_date  Temp_pre_15  Temp_pre_7  \
159    39.965000 -122.318056    CA       8/22/1998    22.357143   21.400000   
192    40.228611 -120.671389    CA       6/27/2006    13.265223   15.781065   
308    39.951177 -122.200397    CA       5/25/2014    20.441860   19.787330   
419    39.936181 -122.093450    CA       8/27/2004     0.000000    0.000000   
453    40.121944 -122.583889    CA       6/18/1996    27.206389   25.695833   
...          ...         ...   ...             ...          ...         ...   
55066  40.343056 -123.383056    CA       7/31/2015    16.234259   16.892857   
55067  40.609722 -123.559444    CA       7/30/2015    16.125000   16.623016   
55070  40.680000 -123.416667    CA       7/30/2015    16.125000   16.623016   
55196  39.911940 -122.493060    CA       7/29/2015    27.802222   27.876543   
55199  40.963600 -120.114200    CA        8/3/2015    20.612145   21.403933   

       Wind_pre_15  Wind_pre_7  Hum_pre_15  Hum_pre

In [11]:
weather_df = pd.read_csv('Weather.csv')
weather_df['fire']=0
print(weather_df)
df = pd.concat([weather_df, fire_df], axis=0)
# print(df)
df = df.sample(frac = 1)
# print(df)
mat=df.corr()
print(mat['fire'])

      latitude   longitude disc_clean_date  Temp_pre_15  Temp_pre_7  \
0    40.343056 -123.383056      2022-10-29         23.0         9.0   
1    40.343056 -123.383056      2022-10-30         22.1         7.3   
2    40.343056 -123.383056      2022-10-31         17.6         9.8   
3    40.343056 -123.383056       2022-11-1         17.0         9.9   
4    40.343056 -123.383056       2022-11-2         17.8         6.7   
..         ...         ...             ...          ...         ...   
339  40.343056 -123.383056       2023-10-3         17.9        10.8   
340  40.343056 -123.383056       2023-10-4         19.1        10.6   
341  40.343056 -123.383056       2023-10-5         14.2        12.2   
342  40.343056 -123.383056       2023-10-6         11.9        11.2   
343  40.343056 -123.383056       2023-10-7         12.9        10.9   

     Wind_pre_15  Wind_pre_7  Hum_pre_15  Hum_pre_7  Prec_pre_15  Prec_pre_7  \
0            3.4        12.5        31.0       69.0          0.0   

In [4]:
from sklearn.model_selection import train_test_split
X = df[['Temp_pre_7', 'Wind_pre_7', 'Hum_pre_7']]
Y = df[['fire']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_test)

C:\Conda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


       Temp_pre_7  Wind_pre_7  Hum_pre_7
25       9.700000    2.900000  36.000000
14535   35.000000    3.100000  12.000000
34151   24.847826    1.251242  42.400621
179      2.800000    9.400000  80.000000
55196   27.876543    4.445679  31.074074
...           ...         ...        ...
327     23.000000    4.000000  33.000000
59       3.000000    2.200000  89.000000
38225  -14.547619    4.173214  81.648810
3611     0.000000    0.000000   0.000000
13332    0.000000    0.000000   0.000000

[205 rows x 3 columns]


In [10]:

# import tensorflow dependencies
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense

# define model architecture
model = Sequential()
model.add(Dense(12, input_dim=3, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=150)

#Test the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# save model and its architecture 
model.save('model.h5')

Epoch 1/150


C:\Conda\lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


26/26 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.6115
Epoch 2/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.4179
Epoch 3/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3689
Epoch 4/150
26/26 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.3566
Epoch 5/150
26/26 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.3480
Epoch 6/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.2880
Epoch 7/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3235
Epoch 8/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 9/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 10/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 

Epoch 80/150
26/26 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 81/150
26/26 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 82/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 83/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 84/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 85/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 86/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 87/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 88/150
26/26 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.3456
Epoch 89/150
26/26 [==============================] - 0s 1ms/ste

C:\Conda\lib\site-packages\tensorflow\python\util\dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)
C:\Conda\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import pickle

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('No Kernnel')
report = classification_report(y_test,y_pred)
# f1scores.append(accuracy_score(y_test,y_pred, average = 'weighted'))
print(report)

In [ ]:
svm_model = svm.SVC(kernel = 'poly')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('No Kernnel')
report = classification_report(y_test,y_pred)
# f1scores.append(accuracy_score(y_test,y_pred, average = 'weighted'))
print(report)

In [ ]:
# With rbf Kernel
svm_model = svm.SVC(kernel = 'rbf')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('rbf Kernel')
print(classification_report(y_test,y_pred))


In [ ]:
# With Linear Kernel
svm_model = svm.SVC(kernel = 'linear')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('Linear Kernel')
print(classification_report(y_test,y_pred))


In [ ]:
# With Sigmoid Kernel
svm_model = svm.SVC(kernel = 'sigmoid')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('Sigmoid Kernel')
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

## With intercept
model = LogisticRegression(fit_intercept=True)

# Fit the model to your training data.
model.fit(X_train,y_train)

# Compute the accuracy of your model on the testing set.
y_pred = model.predict(X_test)

# Print out the report
report = classification_report(y_test,y_pred)
print(report)


In [ ]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

base_classifier = DecisionTreeClassifier(max_depth=1)  # Weak learner for classification
n_estimators = 50  # Number of estimators
ada_classifier = AdaBoostClassifier(base_classifier, n_estimators=n_estimators)
ada_classifier.fit(X_train, y_train)

In [ ]:
y_pred = ada_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
fire_df2 = fire_df[['Temp_pre_7', 'Wind_pre_7','Hum_pre_7','Prec_pre_7', 'fire']]
fire_df2.columns=['Temp', 'Wind', 'Hum', 'Prec', 'fire']
weather_df2 = weather_df[['Temp', 'Wind', 'Hum', 'Prec', 'fire']]
# print(fire_df2)
# print(weather_df2)
df2 = pd.concat([weather_df2, fire_df2], axis=0)
df2 = df2.sample(frac = 1)
mat=df2.corr()
print(mat['fire'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('No Kernnel')
report = classification_report(y_test,y_pred)
# f1scores.append(accuracy_score(y_test,y_pred, average = 'weighted'))
print(report)
